In [1]:
import os, sys, json

autocog_home = os.path.realpath(f"{os.getcwd()}/../../../")
sys.path.append(autocog_home)

from autocog import CogArch
from autocog.utility.models import loader
from autocog.utility.pynb import wrap_graphviz, display

In [2]:
model_names = [
    'llama-2-7b.Q4_K_M',                       # wget https://huggingface.co/TheBloke/Llama-2-7B-GGUF/resolve/main/llama-2-7b.Q4_K_M.gguf
    'llama-2-7b-chat.Q4_K_M',                  # wget https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_K_M.gguf
    'llama-2-13b-chat.Q4_K_M',                 # wget https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_K_M.gguf
    'llama-2-13b-chat.Q5_K_M',                 # wget https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q5_K_M.gguf
    'llama-2-13b-chat.Q8_0',                   # wget https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q8_0.gguf
    'tinyllama-1.1b-chat-v0.3.Q4_K_M',         # wget https://huggingface.co/TheBloke/TinyLlama-1.1B-Chat-v0.3-GGUF/resolve/main/tinyllama-1.1b-chat-v0.3.Q4_K_M.gguf
    'tinyllama-2-1b-miniguanaco.Q4_K_M',       # wget https://huggingface.co/TheBloke/Tinyllama-2-1b-miniguanaco-GGUF/resolve/main/tinyllama-2-1b-miniguanaco.Q4_K_M.gguf
    'capybarahermes-2.5-mistral-7b.Q4_K_M'     # wget https://huggingface.co/TheBloke/CapybaraHermes-2.5-Mistral-7B-GGUF/resolve/main/capybarahermes-2.5-mistral-7b.Q4_K_M.gguf
]
(lm,syntax) = loader(
    # models_path=f"/data/models/{model_names[1]}.gguf",
    prompt_with_format=False,
    prompt_with_index=False,
    prompt_indent='  '
)

In [3]:
arch = CogArch(lm=lm, syntax=syntax)

cog = arch.load('digitize', language='sta', program="""
format digit {
    is enum<width=3>("0","1","2","3","4","5","6","7","8","9");
    annotate f"a single digit";
}

prompt main {
    is {
        number is text;
        digits[3] is digit;
    }
    channel {
        to .number from ?number;
    }
    return {
        from .digits;
    }
}
""")

In [4]:
# dotstr = cog.prompts['main'].instantiate(syntax=arch.syntax, stacks={}, branches={}, inputs={ 'rhs' : [ 1, 2, 3 ], 'lhs': [5,3]}).toGraphViz()
# display(wrap_graphviz(dotstr))

In [5]:
await arch('digitize', number="123")

['6', '6', '2']

In [6]:
display(wrap_graphviz(arch.orchestrator.pages[-1].ftts['main'][-1].toGraphViz(lm)))